# HW5
# Timothy Petliar
# NYU ID: N12592901

The objective of this assignment is use Julia to demonstrate that the ordinary least square estimate for an Ar(1) is downard biased on averaged. The estimate for (alpha, n), where alpha is one of 5 parameters and n s one of 10 samples sizes, is computed 10,000 times.

In [60]:
time()
using PlotlyJS

alpha=[0.5, 0.6, 0.7, 0.8, 0.9]
beta=ones(1,5)
sigma=ones(1,5)
trials=1000

1000

The above code is just the intial values and is is fairly obvious


In [46]:
@everywhere function process_data(sm_x, sm_y ,sm_x_sqrd , sm_x_y, k, a)
    mean_x=sm_x/(k-1)
    mean_y=sm_y/(k-1)
    alpha_est_bias=(sm_x_y-(k-1)*mean_x*mean_y)/(sm_x_sqrd-(k-1)*mean_x*mean_x)-a
    return alpha_est_bias
end

This function will use some inputs computed from the AR1 to actually compute the bias


In [47]:
@everywhere function ar1_ols_data(a,b,s)
   
    sm_x=0 # sum of x_t
    sm_y=0 #sum of x_t+1
    sm_x_sqrd=0 
    sm_x_y=0 # sum of x_t*x_t+1
    y=b/(1-a) #starting value
    results=[]
    for k=1:500  
        x=y
        sm_x+=x
        sm_x_sqrd+=x^2
        y=b+a*y+s*randn()
        sm_y+=y
        sm_x_y+=x*y
        if (k in collect(50:50:500))==true  #Samle sizes less than 500 are computed along the way
            result=process_data(sm_x, sm_y, sm_x_sqrd,sm_x_y,k, a)
            convert(Float64,result)
            i=convert(Int, k/50)
            
            push!(results, result)
        end
    end
    return results
       
end

The function above simulatesthe AR(1) for one set of parameters and computed values relevant to the ols calculation. The relevant values are then passed to Process_Data, returns the bias. An array is returned containing biases for each sample size

In [49]:
@everywhere function repeated_trials(a,b,s,n) #For each set of parameters, simulated 10,000 runs
    sm_results=zeros(10)
    for i=1:n
        results=ar1_ols_data(a,b,s)
        sm_results+=results
    end
    avg_bias=sm_results/n
    return  # computes and return the average of the ols estimated for all sample sizes, but only set of parameters
end

In [63]:
@everywhere function different_params(alpha, beta, sigma, trials)
    global alpha_1;
    global alpha_2    
    global alpha_3
    global alpha_4
    global alpha_5
    t=trials
    alpha_1=repeated_trials(alpha[1], beta[1], sigma[1],t)
    alpha_2=repeated_trials(alpha[2], beta[2], sigma[2],t)
    alpha_3=repeated_trials(alpha[3], beta[3], sigma[3],t)
    alpha_4=repeated_trials(alpha[4], beta[4], sigma[4],t)
    alpha_5=repeated_trials(alpha[5], beta[5], sigma[5],t)
    print(alpha_1)
    return 0
end

 

When the functon above is called, it computed the average for all sets of parameters, by passing to the other functions. 
Each set of parameters is computed separetly and is as separate array, which is easier to work with for plotting. The variables are global, to avoid having to messily return multiple values in some sort of matrix. I think that writing in this form may also allow simple parallel computing.




In [71]:
using PlotlyJS
t0=time()
alpha=[0.5, 0.6, 0.7, 0.8, 0.9]
beta=ones(1,5)
sigma=ones(1,5)
trials=10000


@everywhere function process_data(sm_x, sm_y ,sm_x_sqrd , sm_x_y, k, a)
    mean_x=sm_x/(k-1)
    mean_y=sm_y/(k-1)
    alpha_est_bias=(sm_x_y-(k-1)*mean_x*mean_y)/(sm_x_sqrd-(k-1)*mean_x*mean_x)-a
    return alpha_est_bias
end

@everywhere function ar1_ols_data(a,b,s)
   
    sm_x=0
    sm_y=0
    sm_x_sqrd=0
    sm_x_y=0
    y=b/(1-a)
    results=[]
    for k=1:500
        x=y
        sm_x+=x
        sm_x_sqrd+=x^2
        y=b+a*y+s*randn()
        sm_y+=y
        sm_x_y+=x*y
         if (k in collect(50:50:500))==true
            result=process_data(sm_x, sm_y, sm_x_sqrd,sm_x_y,k, a)
            convert(Float64,result)
            i=convert(Int, k/50)
            
            push!(results, result)
        end
    end
    return results
       
end

@everywhere function repeated_trials(a,b,s,n)
    sm_results=zeros(10)
    for i=1:n
        results=ar1_ols_data(a,b,s)
        sm_results+=results
    end
    avg_bias=sm_results/n
    return avg_bias
end

@everywhere function different_params(alpha, beta, sigma, trials)
    global alpha_1;
    global alpha_2    
    global alpha_3
    global alpha_4
    global alpha_5
    t=trials
    alpha_1=repeated_trials(alpha[1], beta[1], sigma[1],t)
    alpha_2=repeated_trials(alpha[2], beta[2], sigma[2],t)
    alpha_3=repeated_trials(alpha[3], beta[3], sigma[3],t)
    alpha_4=repeated_trials(alpha[4], beta[4], sigma[4],t)
    alpha_5=repeated_trials(alpha[5], beta[5], sigma[5],t)
    return 0
end
 
        
different_params(alpha, beta,sigma, trials)
elapsed=time()-t0
println("time elapsed=", elapsed)
function make_plot()
    x_points=collect(50:50:500)
    trace1 = scatter(;x=x_points, y=alpha_1,mode="lines", name="alpha=0.5")
    trace2 = scatter(;x=x_points, y=alpha_2,mode="lines", name="alpha=0.6")
    trace3 = scatter(;x=x_points, y=alpha_3, mode="lines",name="alpha=0.7")
    trace4 = scatter(;x=x_points, y=alpha_4, mode="lines",name="alpha=0.8")
    trace5 = scatter(;x=x_points, y=alpha_5, mode="lines", name="alpha=0.9")
    
    plt_params = Layout(;title="OLS Estimate Bias",
    xaxis_title="Sample size",
    yaxis_title="Average Bias")
    
    plot([trace1, trace2, trace3, trace4, trace5], plt_params)
    
end

make_plot()

time elapsed=5

.460968017578125


For some strange reason, my Jupyter does not see previous cells, so I'm forced to run the whole chunk of code. As you can see, it took approximetly 5 seconds(It's cut off by the plot). to do all the calculations, which does not seem terrible. In fact, it's pretty similar to the run time I had for the program written in C. The actual results look really good. The ols estimate is both downward biased and increasing in accuracy for all parmater values, as is expected.